### Excel Converersion of Dates from US to Euro Format

In [62]:
import pandas as pd
import os
import re
import openpyxl
from openpyxl.styles import Font, Fill, Color
from openpyxl.styles import colors
from openpyxl.styles import numbers
import numpy as np

from pathlib import Path
from datetime import datetime

#### Change the path where the source files are located:

In [63]:
today = datetime.today()
# sales_file = Path.cwd() / "data" / "raw" / "Sales-History.csv"
# pipeline_file = Path.cwd() / "data" / "raw" / "pipeline_data.xlsx"
# summary_file = Path.cwd() / "data" / "processed" / f"summary_{today:%b-%d-%Y}.pkl"

In [64]:
path = 'C:/Users/bgeorgiev/Python Scripts - ipynb/Euro Dates'
#path = Path.cwd()
convertedpath = path+'/Converted'
#convertedpath = path /'Converted'
os.chdir(path)

In [65]:
USNumberPattern = re.compile(r"""
        ^(\d*)[,]?(\d+)[.]+(\d*)$
        """, re.VERBOSE)

In [66]:
def FromUStoEuroNumber(cell):
    '''used for US format of $$.$$
    '''
    cell = str(cell)
    l = []
    #r = re.search(r'(\d+)[.](\d+)[.,]?(\d*)', cell)
    r = USNumberPattern.search(cell)
    if r:
        #print('r',r.groups())
        if r.group(3):
            l.append('{0}{1},{2}'.format(*r.groups()))
        elif r.group(2):
            l.append('{0},{1}'.format(*r.groups()))
        else:
            l.append('{}'.format(*r.groups()))
        EuroNumber = l[0]
    else:
        EuroNumber = None
    return EuroNumber

In [67]:
FromUStoEuroNumber('700000')
FromUStoEuroNumber('2.9999')
FromUStoEuroNumber('10.99')
FromUStoEuroNumber('12.79999')
FromUStoEuroNumber('100,000.50')

'100000,50'

### A. Function to convert from US YYYYMMDD to Euro Date format

In [68]:
datePatternYYYYMMDD = re.compile(r"""
     ^(.*?) # all text before the date
     ((19|20)\d\d)[- /.] # four digits for the year (must start with 19 or 20)
     ((0|1)?\d)[- /.] # one or two digits for the month
     ((0|1|2|3)?\d) # one or two digits for the day
     (.*?)$ # all text after the date
     """, re.VERBOSE)

In [69]:
def FromUStoEuroDateYYYYMMDD(cell):
    '''used for US format of YYYY-MM-DD
    '''
    cell = str(cell)
    mo = datePatternYYYYMMDD.search(cell)
    #print(cell,'mo',mo.groups())
    if mo == None:
        return None #cell
    else:
    # Get the different parts of the filename.
        beforePart = mo.group(1)
        yearPart   = mo.group(2)
        monthPart  = mo.group(4)
        dayPart    = mo.group(6)
        afterPart  = mo.group(8)

        #Form the European-style filename.
        EuroDate = beforePart + dayPart + '/' + monthPart + '/' + yearPart #+ afterPart
    return EuroDate

In [70]:
# FromUStoEuroDatesYYYYMMDD('2018/11/26')

### B. Function to convert from US MMDDYYYY to Euro Date format

In [71]:
datePatternMMDDYYYY = re.compile(r"""
     ^(.*?) # all text before the date
     ((0|1)?\d)[- /.] # one or two digits for the month
     ((0|1|2|3)?\d)[- /.] # one or two digits for the day
     ((19|20)\d\d) # four digits for the year (must start with 19 or 20)
     (.*?)$ # all text after the date
     """, re.VERBOSE)

In [72]:
def FromUStoEuroDateMMDDYYYY(cell):
    '''used for US format of MM/DD/YYYY
    '''
    cell = str(cell)
    mo = datePatternMMDDYYYY.search(cell)
    #print(cell,'mo',mo.groups())
    if mo == None:
        return None #cell
    else:
    # Get the different parts of the filename.
        beforePart = mo.group(1)
        monthPart   = mo.group(2)
        dayPart  = mo.group(4)
        yearPart    = mo.group(6)
        afterPart  = mo.group(8)

        #Form the European-style filename.
        EuroDate = beforePart + dayPart + '/' + monthPart + '/' + yearPart #+ afterPart
    return EuroDate

In [73]:
# FromUStoEuroDateMMDDYYYY('11/26/1980')
# FromUStoEuroDateMMDDYYYY('abc')

## 2. Iterate through the Source Files, Convert and save a copy in 'Converted' Folder

In [74]:
# Get all the PDF filenames.
ExcelFiles = []
for filename in os.listdir('.'):
    if filename.endswith('.xlsx') and filename.upper().find('EUR')<0:
        #print(filename.upper().find('EUR'))
        ExcelFiles.append(filename)
ExcelFiles.sort()
print('Number of source files in Folder:' , len(ExcelFiles))

Number of source files in Folder: 3


In [75]:
for i, filename in enumerate(ExcelFiles):
    print(i, 'Start: Converting:', filename, '...')
    filename_noExt = (os.path.splitext(os.path.basename(filename))[0])
    wb = openpyxl.load_workbook(filename)
    for worksheet in wb.sheetnames:
        print('...converting:', worksheet, 'tab')
        ws = wb[worksheet] #wb.get_sheet_by_name(worksheet)
        #row_count = sheet.max_row
        #for i in range(row_count):
        for row_cells in ws.iter_rows():
            for cell in row_cells:
                if cell.value is not None:
                    #print('orig:', cell.value)
                    if FromUStoEuroDateMMDDYYYY(cell.value)!= None:
                        #cell.value = FromUStoEuroDateMMDDYYYY(cell.value)
                        cell.font = Font(color=colors.BLUE)
                        cell.number_format = 'text'
                        cell.number_format = 'mm-dd-yy' #'*mm/dd/yyyy'
                    
                    if FromUStoEuroDateYYYYMMDD(cell.value) != None:
                        #cell.value = FromUStoEuroDateYYYYMMDD(cell.value)
                        cell.font = Font(color=colors.BLUE)
                        cell.number_format = 'text'
                        cell.number_format = 'mm-dd-yy' #'*mm/dd/yyyy'
                    
                    if FromUStoEuroNumber(cell.value) != None:
                        #cell.value = FromUStoEuroNumber(cell.value)
                        cell.font = Font(color=colors.BLUE)
                        cell.number_format ='0.00' #'General'
                    #print('new:', cell.value)

    convertedfilename = filename_noExt+'_{}.xlsx'.format('EUR')
    wb.save(convertedpath+'/'+convertedfilename)
    print(i,'End: File coverted to:',convertedfilename )
    print('\n')
print(i+1, 'Files Converted!!!')

0 Start: Converting: DDC_TestData.xlsx ...
...converting: 90_DDC_Trust tab
...converting: 91_DDC_Tranche tab
...converting: 92_DDC_CashAccount tab
...converting: 93_DDC_Hedge tab
...converting: 94_DDC_Group tab
...converting: 95_DDC_Fees tab
...converting: 96_DDC_CreditEnhancement tab
...converting: 97_DDC_Deal tab
...converting: 98_DDC_Events tab
...converting: 99_DDC_Class tab
0 End: File coverted to: DDC_TestData_EUR.xlsx


1 Start: Converting: ManualInput_TestData.xlsx ...
...converting: 100_MI_ActualIndexRates tab
...converting: 101_MI_CollateralAdjustments tab
...converting: 102_MI_CPExpense tab
...converting: 103_MI_InvestmentIncome tab
...converting: 104_MI_FeeExpenses tab
...converting: 105_MI_TrancheBalanceChanges tab
1 End: File coverted to: ManualInput_TestData_EUR.xlsx


2 Start: Converting: Stratification_TestData.xlsx ...
...converting: 01_Strat_StratificationGroup tab
...converting: 02_Strat_Stratification tab
...converting: 03_Strat_StratificationLevel tab
2 End: File 